In [1]:
import pandas as pd 

products_df = pd.read_pickle('./hackathon-inditex-data-recommender/data/raw/products.pkl')

train_df = pd.read_csv('./hackathon-inditex-data-recommender/data/raw/train.csv')

In [2]:
products_df.head()

,discount,embedding,partnumber,color_id,cod_section,family
0,0,"[-0.13401361, -0.1200429, -0.016117405, -0.167...",32776,85,4.0,73
1,0,"[-0.0949274, -0.107294075, -0.16559914, -0.174...",41431,135,4.0,73
2,0,"[-0.12904441, -0.07724628, -0.09799071, -0.164...",39419,339,4.0,73
3,1,"[-0.12783332, -0.133868, -0.10101265, -0.18888...",36087,135,4.0,73
4,1,"[-0.14092924, -0.1258284, -0.10809927, -0.1765...",34132,3,4.0,73


In [4]:
train_df.head()

,session_id,date,timestamp_local,add_to_cart,user_id,country,partnumber,device_type,pagetype
0,64,2024-06-06,2024-06-06 16:43:17.389,0,NaN,29,14327,1,24.0
1,117,2024-06-08,2024-06-08 15:11:02.782,0,NaN,57,38422,1,24.0
2,117,2024-06-08,2024-06-08 15:11:44.797,0,NaN,57,19763,1,24.0
3,579,2024-06-05,2024-06-05 19:24:48.397,0,NaN,29,30253,1,24.0
4,1220,2024-06-04,2024-06-04 08:21:13.476,0,480729.0,25,1592,1,24.0


In [3]:
#query 1:
df_filtro = products_df[(products_df['color_id'] == 3) & (products_df['discount'] == 1)]

min_family = df_filtro.loc[df_filtro['family'].idxmin()]

result_partnumber = min_family['partnumber']

print(result_partnumber)

17265


In [7]:
#query2:
df = train_df.dropna(subset=['user_id', 'add_to_cart'])

user_purchase_counts = df[df['add_to_cart'] == 1].groupby('user_id').size()

low_purchase_users = user_purchase_counts[user_purchase_counts < 500].index

df_filtered = df[df['user_id'].isin(low_purchase_users)]

country_counts = df_filtered['country'].value_counts()

target_country = country_counts.idxmax()

df_country_users = df_filtered[df_filtered['country'] == target_country]

user_frequency = df_country_users.groupby('user_id').size()

df_country_users['timestamp_local'] = pd.to_datetime(df_country_users['timestamp_local'])

most_recent_purchase = df_country_users.groupby('user_id')['timestamp_local'].max()

user_data = pd.DataFrame({
    'frequency': user_frequency,
    'most_recent_purchase': most_recent_purchase
})

df_user_data = df_country_users.merge(user_data, left_on='user_id', right_index=True)

df_user_data = df_user_data.sort_values(by=['frequency', 'most_recent_purchase', 'user_id'], ascending=[True, False, True])

result_user = df_user_data.iloc[0]

print(result_user['user_id'])


/tmp/ipykernel_2659387/1956789078.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country_users['timestamp_local'] = pd.to_datetime(df_country_users['timestamp_local'])


137495.0


In [11]:
#query3 
df = train_df.copy()

df['timestamp_local'] = pd.to_datetime(df['timestamp_local'])

df_added_to_cart = df[df['add_to_cart'] == 1]

df_sorted = df.sort_values(by=['partnumber', 'timestamp_local'])

df_sorted['cart_added'] = df_sorted['add_to_cart'].cumsum()

def count_visits(group):
    group['visits_before_add'] = group['cart_added'].shift(fill_value=0)
    return group

df_sorted = df_sorted.groupby('partnumber').apply(count_visits)

df_added_to_cart_sorted = df_sorted[df_sorted['add_to_cart'] == 1]

average_visits = df_added_to_cart_sorted['visits_before_add'].mean()
average_visits

/tmp/ipykernel_2659387/3692466831.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted = df_sorted.groupby('partnumber').apply(count_visits)


1371804.1581283007

In [14]:
#query4
df_merged = pd.merge(train_df, products_df, on='partnumber', how='inner')

df_filtered = df_merged[(df_merged['discount'] == 1) & (df_merged['add_to_cart'] == 1)]

device_usage = df_filtered['device_type'].value_counts()

device_usage

most_frequent_device = device_usage.idxmax()

print(most_frequent_device)


device_type
1    169439
3     16416
2      1884
Name: count, dtype: int64

In [ ]:
#query5
device3 = train_df[train_df['device_type'] == 3]

user_int = device3.groupby('user_id')['partnumber'].nunique()

df_freq = train_df.groupby(['user_id', 'country'])['add_to_cart'].sum().reset_index()

top_users_by_country = df_freq.groupby('country').apply(lambda x: x.nlargest(3, 'add_to_cart')).reset_index(drop=True)

top_users = top_users_by_country['user_id'].unique()

df_top_users_device_3 = device3[device3['user_id'].isin(top_users)]

user_with_most_interactions = user_int.loc[df_top_users_device_3['user_id'].value_counts().idxmax()]

print(user_with_most_interactions)


49


/tmp/ipykernel_2659387/973678857.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_users_by_country = df_freq.groupby('country').apply(lambda x: x.nlargest(3, 'add_to_cart')).reset_index(drop=True)


In [23]:
#query 6
df = train_df.copy()
user_countries = df.groupby('user_id')['country'].apply(set)

df['is_outside_country'] = df.apply(
    lambda row: row['country'] not in user_countries[row['user_id']] if pd.notna(row['country']) and pd.notna(row['user_id']) else False,
    axis=1
)

df_outside_country = df[df['is_outside_country']]

df_merged = pd.merge(df_outside_country, products_df[['partnumber', 'family']], on='partnumber', how='left')

unique_families_count = df_merged['family'].nunique()
unique_families_count

0

In [28]:
#query7
df = train_df.copy()
products = products_df.copy()

df_merged = pd.merge(df, products[['partnumber', 'family']], on='partnumber', how='left')

df_merged['date'] = pd.to_datetime(df_merged['date'])  
df_filtered = df_merged[(df_merged['date'] >= '2024-06-01') & (df_merged['date'] <= '2024-06-07')]

df_filtered_cart = df_filtered[df_filtered['add_to_cart'] == 1]

page_frequency = df_filtered_cart.groupby(['family', 'pagetype']).size().reset_index(name='count')

most_frequent_page = page_frequency.loc[page_frequency.groupby('family')['count'].idxmax()]

most_frequent_page = most_frequent_page.loc[most_frequent_page.groupby('family')['pagetype'].transform('min') == most_frequent_page['pagetype']]

result = {str(int(row['family'])): int(row['pagetype']) for _, row in most_frequent_page.iterrows()}

# Mostrar el resultado con el formato exacto y sin decimales
import json
print(json.dumps(result, indent=4))

{
    "1": 24,
    "2": 24,
    "3": 24,
    "4": 24,
    "5": 24,
    "6": 24,
    "7": 24,
    "8": 24,
    "9": 24,
    "10": 24,
    "11": 24,
    "12": 24,
    "13": 24,
    "14": 24,
    "15": 24,
    "16": 24,
    "17": 24,
    "18": 24,
    "19": 24,
    "21": 24,
    "22": 24,
    "23": 24,
    "24": 24,
    "26": 24,
    "27": 24,
    "28": 24,
    "29": 24,
    "30": 24,
    "31": 24,
    "32": 24,
    "33": 24,
    "34": 24,
    "35": 24,
    "36": 24,
    "37": 24,
    "38": 24,
    "40": 24,
    "41": 24,
    "42": 24,
    "43": 24,
    "44": 24,
    "45": 24,
    "46": 24,
    "47": 24,
    "48": 24,
    "49": 24,
    "50": 24,
    "51": 24,
    "52": 24,
    "53": 24,
    "54": 24,
    "56": 24,
    "57": 24,
    "58": 24,
    "59": 24,
    "60": 24,
    "61": 24,
    "62": 24,
    "63": 24,
    "64": 24,
    "65": 24,
    "66": 24,
    "67": 24,
    "68": 24,
    "69": 24,
    "70": 24,
    "71": 24,
    "72": 24,
    "73": 24,
    "74": 24,
    "75": 24,
    "76": 24,